In [2]:
import os
import pandas as pd
from tqdm import tqdm
import glob
from datetime import datetime

In [3]:
# !pip install pyarrow

In [6]:
PARENT_DIR = os.path.dirname(os.getcwd())
# CSV FILE
CSV_FOLDER = os.path.join(PARENT_DIR,'csv_files')

# PQ FILE
PQ_FOLDER = os.path.join(PARENT_DIR,'pq_files')

# XLSX FILE
XLSX_FOLDER = os.path.join(PARENT_DIR,'xlsx_files')

os.makedirs(CSV_FOLDER,exist_ok=True) #csv파일 저장 폴더 생성
os.makedirs(PQ_FOLDER,exist_ok=True) #csv파일 저장 폴더 생성

In [12]:
XLSX_FOLDER = os.path.join(PARENT_DIR,'xlsx_files')
excel_files = glob.glob(XLSX_FOLDER+'\\*.xlsx')
for file in tqdm(excel_files):
    print(file)
    break

  0%|          | 0/262 [00:00<?, ?it/s]

c:\Users\samle\Documents\DevCourse\xlsx_files\KOBIS_일별_박스오피스_2024-12-28 (1).xlsx


In [24]:
#string에서 Datetime 추출
date_str_dt = datetime.strptime(date_str,"%Y%m%d")
date_str_dt
df_curdate2 = pd.DataFrame({'cur_date':[date_str_dt]*len(data)})
df_curdate2

,cur_date
0,2024-12-16
1,2024-12-16
2,2024-12-16
3,2024-12-16
4,2024-12-16
...,...
120,2024-12-16
121,2024-12-16
122,2024-12-16
123,2024-12-16


In [13]:
excel_files = glob.glob(os.path.join(XLSX_FOLDER,'*.xlsx')) #xlsx 파일 리스트

for file in tqdm(excel_files):
    df = pd.read_excel(file, skiprows=4, engine="openpyxl")
    #순위가 포함된 행들찾기
    rank_rows = df[df.apply(lambda row: row.astype(str).str.contains('순위').any(), axis=1)].index
    #합계가 포함된 행들 찾기기
    gross_total_rows = df[df.apply(lambda row: row.astype(str).str.contains('합계').any(), axis=1)].index
    #DataFrame에 사용할 컬럼명 추출
    columns = df.iloc[rank_rows[0]].values.tolist()
    new_cols = []

    #향후 SQL 데이터 처리를 용이하기 위해 컬럼명 변경
    for col in columns:
        col = col.strip() #Trailing space 삭제
        if '(' in col:
            col = col[:-1] # ')' 제거
            new_col = col.replace('(','_')
            new_cols.append(new_col)
        else:
            new_cols.append(col)
    #엑셀파일에서서 일별 데이터 추출 및 저장 (CSV 및 parquet으로 저장)
    for i, rank_row in enumerate(rank_rows):
        date = df.iloc[rank_row-1][0][1:-3]# 날짜 정보만 추출출
        dates = date.split()
        date_str = ''
        for date in dates:
            date_str += date[:-1]#년,월,일 제거후 YYYYMMDD형식으로 변경경
        data = df.iloc[rank_row+2:gross_total_rows[i],:].values
        df_daily = pd.DataFrame(data,columns=new_cols)
        df_daily.columns.values[4] = '매출액_점유율' #2번째'매출액'에서 '매출액_점유율'로 변경
        df_daily.columns.values[5] = '매출액증감_전일대비' # 매출액증감에서 '매출액증감_전일대비'로 변경
        df_vote_date = pd.DataFrame({'voting_date':[date_str]*len(data)})#조회 일자 column 생성
        df_vote_date['voting_date'] = pd.to_datetime(df_vote_date['voting_date'], format="%Y-%m-%d")# sring에서 datetime으로 변경
        df_final = pd.concat([df_vote_date,df_daily],axis=1)# 조회일자 합치기
        df_final['영화명']=df_final['영화명'].apply(lambda x: str(x) if isinstance(x,(int,float)) else x) #영화명이 숫자로만 이루어져 있을 경우 string으로 전환
        df_final.to_parquet(os.path.join(PQ_FOLDER,f'{date_str}.parquet'),engine='pyarrow',compression='snappy') #추출한 날짜를 파일명으로 하고 Parquet으로 저장
        df_final.to_csv(os.path.join(CSV_FOLDER,f'{date_str}.csv'),header=True,index=False,encoding='utf-8-sig')#추출한 날짜를 파일명으로 하고 한글깨짐 방지를 위해 utf-8-sig로 encoding후 CSV로 저장


100%|██████████| 262/262 [02:05<00:00,  2.08it/s]
